In [11]:
#import findspark
#findspark.init('/home/max/Downloads/spark-2.4.0-bin-hadoop2.7')
# from pyspark.sql import SparkSession
# from pyspark.sql import Row



In [2]:
#spark= SparkSession.builder.getOrCreate()

In [149]:
from io import StringIO
import matplotlib.pyplot as plt
import random
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta
import csv
import sys
from pympler.asizeof import asizeof
import boto3
import time

In [4]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)

In [5]:
# import code counts, and convert to dict with probabilities
code_counts = pd.read_csv('code_counts.csv')
code_counts.columns = ['code', 'count']
code_counts['prop'] = code_counts['count']/code_counts['count'].sum()
#code_counts.head()
code_counts = dict(zip(code_counts['code'], code_counts['prop']))


In [6]:
# get random code based on the weight (probability) of each code
def get_random_code(dct):
    rand_val = random.random()
    total = 0
    for k, v in dct.items():
        total += v
        if rand_val <= total:
            return k
    assert False, 'unreachable'
    
get_random_code(code_counts)

'C02'

In [127]:
filesize = 120000000 # 120mb
max_size = 1000000000 # 1gb
total_files = max_size/filesize
print(int(total_files))

8


In [161]:
start_value = 5
start_time_shift =20
random_walk_weight = 0.01
random_walk_variance = 1
seasonality_1_freq = 50 #lower is shorter
seasonality_1_weight = 5
seasonality_2_freq = 7 #lower is shorter
seasonality_2_weight = 2

num_words_in_text = 100 # number of words in each patent text

sample_size = 1000

trend = 0.02
exponent = 1
n_samples = 10
bucket_name = 'patents-bucket'

times = [0,0,0,0,0,0]

terms = []
with open('wordlist.csv', 'r') as f:
    reader = csv.reader(f)
    terms = [word[0] for word in list(reader)]
    random.shuffle(terms)
        
# store dataframe to S3 in CSV format
def store_data(df, file_num):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name, 'patent-100mbs/patents_'+str(file_num)+'.csv').put(Body=csv_buffer.getvalue())

# Get number of samples to be made for one day
def get_num_samples(base):
    #Assume no start at y = 0
    random_walk = np.random.normal(0,random_walk_variance)
    time = base + start_time_shift

    y = (np.sin(time/seasonality_1_freq))*seasonality_1_weight #Add seasonality 1
    y = y + np.sin(time/seasonality_2_freq)*seasonality_2_weight #Add seasonality 2
    y = y + (trend * (time ** exponent)) #Add trend

    # Random walk component
    random_walk += np.random.normal(0,random_walk_variance)
    
    return max(0, y + random_walk * random_walk_weight)


def create_datasets(filesize, max_files):
    print("Storing", filesize/1000000, "MB in each file, in a total of", max_files, "files")
    rows = []
    sample_range = 0
    file_num = 0
    
    i = 0
    while file_num < max_files:

        if sys.getsizeof(rows) > filesize: # larger than filesize (120mb?), store file
            df = pd.DataFrame(rows, columns=['date', 'code', 'text'])
            #print("stored", sys.getsizeof(rows)/1000000, "MB in file num", file_num)
            store_data(df, file_num)
            print("stored", sys.getsizeof(rows)/1000000, "MB in file num", file_num)
            rows = []
            file_num+=1
            
        num_samples = get_num_samples(i)
        
        for n in range(int(round(num_samples))): #Create n entries
            date = datetime.datetime(1950, 1, 1, 0, 0) + timedelta(days=int(i))
            current_sample = i+n
            #sample = terms[current_sample:current_sample+100]
            #text = " ".join(random.sample(terms, num_words_in_text)) # TOO SLOW
            rnd = random.randint(1,len(terms)-101)
            text = " ".join(terms[rnd:rnd+100]) 
            rows.append([date, get_random_code(code_counts), text])
        
        
        
        i+=1

filesize = 120000000 # 120mb
max_total_size = 2000000000 # 5gb
#filesize = 1200 #
#max_total_size = 4000 #
max_files = int(max_total_size/filesize)
#filesize=12000
create_datasets(filesize, max_files)


Storing 120.0 MB in each file, in a total of 16 files


KeyboardInterrupt: 

77